# Setup 

In [1]:
# setup
import os
import sys

# para importar módulos en '../../tools'
module_path = os.path.abspath(os.path.join('../../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import locale

import re
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from pylab import *
import matplotlib.gridspec as gridspec

import seaborn as sns
from urllib.parse import parse_qs

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import maxabs_scale, minmax_scale, MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
# import squarify

import json

import analytics_tools
import requests
from datetime import datetime, date, timedelta

locale.setlocale(locale.LC_ALL, '')

%matplotlib inline

sns.set()
plt.rcParams['image.cmap'] = 'Accent'

pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', -1)
pd.set_option('precision',2)
pd.set_option('display.float_format', lambda x: locale.format_string('%d', x, 1))


In [2]:
# actualizo los analytics de ambas apis
analytics_tools.update_analytics()

In [3]:
# formarters
f_ar = lambda x: locale.format_string('%.2f', x, 1)
d_ar = lambda x: locale.format_string('%d', x, 1)

tz_arg = lambda x: pd.to_datetime(x).tz_localize('UTC').tz_convert(tz='America/Argentina/Buenos_Aires')

TABLE_COUNTER = 0
TABLE_TEMPLATE = """
<center><strong><small>{title}</small></strong></center>
<center>{table}</center>
<center><strong><small>Tabla {table_number}</small></strong></center>
"""
    
def table_counter():
    global TABLE_COUNTER
    TABLE_COUNTER += 1
    return TABLE_COUNTER

def add_style_to_df(df, subset, color='black', font_weight=None):
    render = df.style.set_properties(
        subset= subset, 
        **{'font-weight': font_weight, 'color':color}).render().replace('\n','')
    return render

def add_title(df_html, title):
    str_table = TABLE_TEMPLATE.format(title=title, table=df_html, table_number=table_counter())
    return str_table.replace('\n','')

def put_df_on_report(df, title, subset=None, color='black', font_weight='bold'):
    if subset:
        df_html = add_style_to_df(df, subset=subset,color=color, font_weight=font_weight)
    else:
        df_html = df.to_html().replace('\n','')
    return add_title(df_html, title)

In [4]:
midpoint = lambda x: (np.max(x)+ np.min(x))/2
log_midpoint = lambda x: np.power(10,((np.log10(np.max(x))+np.log10(np.min(x)))/2))

def read_files_to_df(directory):
    """Lee CSVs de misma estructura en un directorio a un solo DataFrame."""

    file_pattern = os.path.join(directory, "*.csv")
    dfs = [pd.read_csv(file, encoding="utf8", parse_dates=True)
           for file in glob.glob(file_pattern)]
        
    return pd.concat(dfs, axis=0)

def parse_ids(qs):
    if pd.notna(qs):
        if 'ids' in qs:
            params = parse_qs(qs)
            ids_values_str = params.get('ids',[''])[0]
        else:
            ids_values_str = qs

        return [value.split(':')[0] for value in ids_values_str.split(',')]
    else:
        return []

def iter_series_ids(df):
    for analytic_id, id_list in zip(df.id, df.ids):
            if isinstance(id_list, list):
                for serie_id in id_list:
                    yield (analytic_id, serie_id)
    
def unfold_series_id(df):
    list_ids = []
    for analytic_id, serie_id in iter_series_ids(df):
        list_ids.append([analytic_id, serie_id])
        
    df_ids = pd.DataFrame(list_ids, columns=['analytic_id','serie_id'])
    df_ids.dropna(inplace=True)
    
    df_ids['analytic_id'] = df_ids['analytic_id'].astype(object)
    
    df_extended = df_ids.merge(df, left_on='analytic_id', right_on='id')
    df_extended = df_extended.drop(columns=['analytic_id'])
    
    return df_extended

def replace_and_drop_column(df, old_column, new_column):
    df[new_column] = df[old_column]
    new_df = df.drop(old_column, axis=1)
    return new_df

def add_totals(df):
    df_to_append = pd.DataFrame(df.sum(numeric_only=True)).T
    
    df_appended = df.append(df_to_append)
    
    as_list = df_appended.index.tolist()
    idx = as_list.index(0)
    as_list[idx] = 'Total'
    df_appended.index = as_list
    
    return df_appended

def add_linear_regression(df, target_col, index_as_feature=True, feature_cols=''):
    if index_as_feature:
        X = df.index.values.reshape(-1, 1)
    else:
        n_r, n_c = df[feature_cols].shape
        X = df[feature_cols].values.reshape(-1, n_c)
        
    y = df[target_col].values
    
    reg = LinearRegression()
    reg.fit(X, y)
    
#     df['linear_regression'] = 
    return reg.predict(X)

#
def interpolate_by_days(df, column, date_col='' ,days=1, floor=0):
    df['day_factor'] = pd.DatetimeIndex(df[date_col]).dayofyear % days
    day_factor_unique = df.day_factor.unique()
    list_df = []

    call_col_inter = column + '_inter'
    
    for day_factor in day_factor_unique:
#         for call_col in columns:
        
        filter_by_df = df.day_factor == day_factor
        df_f = df[filter_by_df]
        df_f[call_col_inter] = df_f[column].apply(lambda x: np.nan if x < floor else x)
        df_f.loc[:,call_col_inter] = df_f[call_col_inter].interpolate(method='linear', limit_direction='both',inplace=False)
        list_df.append(df_f)
        del df_f

    df_inter = pd.concat(list_df, sort=True)
    
    return df_inter[call_col_inter]

In [5]:
# para retención de usr
def get_week(d):
    start = d - pd.Timedelta(days=d.weekday())
    end = start + pd.Timedelta(days=6)
    return end

def cohort_period(df):
    df['cohort_period'] = np.arange(len(df)) + 1
    return df

In [6]:
# para clusterizar
# recibe un unfolded
def get_nunique_series_avg(df, frequency='W'):
    """Devuelve el promedio de cantidad de series únicas por la frencuencia indicada """
    df_list = []
    
    for frequency in 'DWM':
        if frequency is 'D':
            col = 'date'
            frec_col = 'series_unicas_diarias'
        elif frequency is 'W':
            col = 'weekofyear'
            frec_col = 'series_unicas_semanales'
        elif frequency is 'M':
            col = 'month'
            frec_col = 'series_unicas_mensuales'
        else:
            return
        
        df_avg_series = df.groupby(['ip_address',col]).nunique()['serie_id'].reset_index()
        
#         n_series_unicas = df_avg_series.serie_id.count()
        
        df_avg_series = df_avg_series[['ip_address', 'serie_id']].groupby('ip_address').mean()
        df_avg_series.rename({'serie_id':frec_col},axis=1, inplace=True) 
                
        df_list.append(df_avg_series)

    df_series_avg = pd.concat(df_list, axis=1).reset_index()
    
    ass = df[['month','ip_address', 'serie_id','indice_tiempo_frecuencia']].drop_duplicates()
    id_unicos_por_ip_mes = ass.groupby(['ip_address','month']).nunique()['serie_id'].reset_index()
    
    serie_diaria = ass.indice_tiempo_frecuencia.str.contains('R/P1D')
    id_diarios_unicos_por_ip_mes = ass[serie_diaria].groupby(['ip_address','month']).nunique()['serie_id'].reset_index()
    
#     ### ip - month - su - sud
#     ass = df[['date_short','ip_address', 'serie_id','indice_tiempo_frecuencia']].drop_duplicates()
#     id_unicos_por_ip_mes = ass.groupby(['ip_address','date_short']).nunique()['serie_id'].reset_index()
#     id_unicos_por_ip_mes.rename({'serie_id':'series_unicas'}, axis=1, inplace=True)
#     serie_diaria = ass.indice_tiempo_frecuencia.str.contains('R/P1D')
#     id_diarios_unicos_por_ip_mes = ass[serie_diaria].groupby(['ip_address','date_short']).nunique()['serie_id'].reset_index()
#     id_diarios_unicos_por_ip_mes.rename({'serie_id':'series_diarias_unicas'}, axis=1, inplace=True)

#     #n_id_diarios_unicos = id_diarios_unicos_por_ip_mes.groupby('ip_address').mean().reset_index()
#     asss = id_unicos_por_ip_mes.merge(id_diarios_unicos_por_ip_mes, on=['ip_address','date_short'], how='left').fillna(0)

#     ####
#     assss = asss[['ip_address','series_unicas','series_diarias_unicas']].groupby('ip_address').mean()

    ### ip - su - sud
    ass = df[['ip_address', 'serie_id','indice_tiempo_frecuencia']].drop_duplicates()
    id_unicos_por_ip_mes = ass.groupby('ip_address').nunique()['serie_id'].reset_index()
    id_unicos_por_ip_mes.rename({'serie_id':'series_unicas'}, axis=1, inplace=True)
    serie_diaria = ass.indice_tiempo_frecuencia.str.contains('R/P1D')
    id_diarios_unicos_por_ip_mes = ass[serie_diaria].groupby('ip_address').nunique()['serie_id'].reset_index()
    id_diarios_unicos_por_ip_mes.rename({'serie_id':'series_diarias_unicas'}, axis=1, inplace=True)

    #n_id_diarios_unicos = id_diarios_unicos_por_ip_mes.groupby('ip_address').mean().reset_index()
    asss = id_unicos_por_ip_mes.merge(id_diarios_unicos_por_ip_mes, on='ip_address', how='left').fillna(0)

    ####
    assss = asss

    df_series_avg = df_series_avg.merge(assss, on='ip_address').fillna(0)

    return df_series_avg


# recibe un unfolded!
def get_ips_calls_max(df, frequency='D'):
    """Devuelve el promedio diario de llamadas por IP """
    if frequency is 'W':
        df_ncalls = df.groupby(['ip_address','weekofyear']).nunique()['id'].reset_index()
    elif frequency is 'D':
        df_ncalls = df.groupby(['ip_address','date']).nunique()['id'].reset_index()
    else:   
        return
    
    df_ncalls = df_ncalls.groupby(['ip_address']).max().reset_index()
    df_ncalls.rename({'id':'max_calls'},axis=1, inplace=True)
    df_ncalls.sort_values('max_calls', ascending=False, inplace=True)

    return df_ncalls

# recibe un unfolded!
def get_ips_calls_count(df, frequency='D', scale='linear'):
    """Devuelve el promedio diario de llamadas por IP """
    if frequency is 'W':
        df_ncalls = df.groupby(['ip_address','weekofyear']).nunique()['id'].reset_index()
    elif frequency is 'D':
        df_ncalls = df.groupby(['ip_address','date']).nunique()['id'].reset_index()
    else:   
        return
    
    df_ncalls = df_ncalls.groupby(['ip_address']).mean().reset_index()
    df_ncalls.rename({'id':'actividad'},axis=1, inplace=True)
    df_ncalls.sort_values('actividad', ascending=False, inplace=True)

    if scale is 'log':
        df_ncalls['actividad'] = df_ncalls.actividad.apply(np.log10)
    
    return df_ncalls

# recibe un unfolded!
def get_ips_persistency(df, frequency='W', scale='linear'):
    """Devuelve la proporción de semanas con actividad para cada IP
    Args:
        frequency: str. W o D
    """
    if frequency is 'W':
#         group_cols = ['year','month','ip_address']
#         data_cols = group_cols + ['weekofyear']
#         n_frec = df[['year','month','weekofyear']].groupby(['year','month']).nunique()[['weekofyear']].sum().values[0]
#         df_persistency = df[data_cols].groupby(group_cols).nunique()[['weekofyear']]

        group_cols = ['ip_address']
        data_cols = group_cols + ['weekdate']
        n_frec = df['weekdate'].nunique()
        df_persistency = df[data_cols].groupby(group_cols).nunique()[['weekdate']]
        
#         print(n_frec)

    elif frequency is 'D':
        group_cols = ['ip_address']
        data_cols = group_cols + ['date']

        n_frec = df.date.nunique()

        df_persistency = df[data_cols].groupby(group_cols).nunique()[['date']]
    else:   
        return
    
    df_persistency = df_persistency.unstack(level=0).unstack(level=0)
    df_persistency = df_persistency.fillna(0).sum(1).reset_index()
    df_persistency.rename({0:'persistencia'},axis=1, inplace=True)
    df_persistency.sort_values('persistencia', ascending=False, inplace=True)
    df_persistency['persistencia'] = df_persistency.persistencia.divide(n_frec/100)
    
    if scale is 'log':
        df_persistency['persistencia'] = df_persistency.persistencia.apply(np.log10)

    return df_persistency

# recibe un unfolded!
def get_ip_features(df, frequencies={'persistencia':'W','actividad':'D'}, scales={'persistencia':'linear','actividad':'log'}):
    
    df_ncalls = get_ips_calls_count(df, frequency=frequencies['actividad'], scale=scales['actividad'])
    df_persistency = get_ips_persistency(df, frequency=frequencies['persistencia'], scale=scales['persistencia'])
#     df_maxcalls = get_ips_calls_max(df)

    df_ip_features = df_ncalls.merge(df_persistency, how='inner', on='ip_address')
#     df_ip_features = df_ip_features.merge(df_maxcalls, how='inner', on='ip_address')
    
    return df_ip_features

# 
def cluster_ips(df, n_kmeans=3, frequencies={'persistencia':'W','actividad':'D'}, scales={'persistencia':'linear','actividad':'log'},labels=None):
    df_ip_features = get_ip_features(df, frequencies, scales)

    X = df_ip_features[['actividad','persistencia']].values.reshape(-1, 2)
    X_minmax = MinMaxScaler().fit_transform(X)
#     X_minmax = StandardScaler().fit_transform(X)
    
    clusters_id = KMeans(n_kmeans, random_state=0).fit_predict(X_minmax)

    ips_clusterizados = df_ip_features.merge(pd.DataFrame(clusters_id), how='inner', left_index=True, right_index=True)
    ips_clusterizados.rename({0:'cluster_id'}, axis=1, inplace=True)
    ips_clusterizados['cluster_name'] = ips_clusterizados.cluster_id.apply(lambda x: labels[x])
    return ips_clusterizados


In [7]:
from pylab import *
import matplotlib.gridspec as gridspec

def make_cluster_plots(df_cluster, clusters, exclude_clusters_id=None):
    
    G = gridspec.GridSpec(4, 3)

    plt.figure(figsize=[15,20])
    
    axes_1 = subplot(G[0:2, :])
    plt.scatter(df_cluster['actividad'], df_cluster['persistencia'], c=df_cluster['color'], s=50,alpha=.7);
    plt.title('Segmentación de usuarios, período completo')
    plt.xlabel('actividad')
    plt.ylabel('persistencia');

    axes_3 = subplot(G[2:,:-1])
    act_std = df_cluster.actividad.std()
    pers_std = df_cluster.persistencia.std()
    act_mean = df_cluster.actividad.mean()
    pers_mean = df_cluster.persistencia.mean()

    df_cluster['actividad_norm'] = minmax_scale(df_cluster.actividad, feature_range=(act_mean-2*act_std,act_mean+2*act_std))
    df_cluster['persistencia_norm'] = minmax_scale(df_cluster.persistencia, feature_range=(pers_mean-2*pers_std,pers_mean+2*pers_std))
    
    df_cluster_profile = df_cluster[~df_cluster.cluster_id.isin(exclude_clusters_id)].groupby(['cluster_id','cluster_name','color']).agg({'ip_address': pd.Series.nunique,
                                            'actividad_norm': pd.Series.mean,
                                            'persistencia_norm': pd.Series.mean}).reset_index()

    df_cluster_profile = df_cluster_profile.sort_values('ip_address', ascending=False)

    with sns.axes_style('white'):
    #     fig, ax = plt.subplots(figsize=(15,10))
        axes_3.spines['bottom'].set_position('center')
        axes_3.spines['left'].set_position('center')
        axes_3.spines['bottom'].set_color('grey')
        axes_3.spines['left'].set_color('grey')
        axes_3.spines['right'].set_color('none')
        axes_3.spines['top'].set_color('none')
        axes_3.xaxis.set_label_coords(.95, 0.48)
        axes_3.yaxis.set_label_coords(0.53, 0.08)
        axes_3.set_alpha(.2)

        a = np.abs(df_cluster_profile.actividad_norm)
        p = np.abs(df_cluster_profile.persistencia_norm)
        u = df_cluster_profile.ip_address
        cluster_s = u * a

        df_cluster_profile.plot.scatter('actividad_norm','persistencia_norm', 
                                    c=df_cluster_profile.color.values,
#                                     c=df_cluster_profile.index.values,
                                    s=cluster_s, 
                                    cmap='Accent', 
                                    colorbar=False,
                                    alpha=.55,
                                    legend=True,
                                    ax=axes_3);

    for i, row in df_cluster_profile.iterrows():
        txt = row.cluster_name
        x = row.actividad_norm
        y = row.persistencia_norm
        axes_3.text(x, y, txt, horizontalalignment='center',verticalalignment='bottom')

    plt.xticks([])
    plt.yticks([])

    plt.xlabel('actividad');
    plt.ylabel('persistencia');
    plt.title('Variables normalizadas.');

    df_cluster_agg = df_cluster.groupby(['cluster_id','cluster_name','color']).agg({'ip_address': pd.Series.nunique,
                                                           'actividad': pd.Series.sum}).reset_index()
    df_cluster_agg = df_cluster_agg.sort_values('cluster_id')

    col = 'ip_address'
    title = 'Usuarios'
    axes = subplot(G[2,-1])


    patches = df_cluster_agg[col].plot(kind='pie',autopct='%.1f',labels=None, fontsize=0, colors=df_cluster_agg.color.values, ax=axes)
    plt.title(title)
    plt.ylabel('');
    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(df_cluster_agg.cluster_name, df_cluster_agg[col].divide(df_cluster_agg[col].sum())*100)]
    plt.legend(labels=labels, loc='best')

    col = 'actividad'
    title = 'Llamadas'
    axes = subplot(G[3,-1])

    patches = df_cluster_agg[col].plot(kind='pie',autopct='%.1f',labels=None, fontsize=0, colors=df_cluster_agg.color.values, ax=axes)
    plt.title(title)
    plt.ylabel('');
    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(df_cluster_agg.cluster_name, df_cluster_agg[col].divide(df_cluster_agg[col].sum())*100)]
    plt.legend(labels=labels, loc='best')
        
    plt.savefig('../graphs/graphs-{}-clusters.png'.format(clusters), dpi=64)
    plt.show()
    
    

In [8]:
# esto necesita un superrefactor

def pie_plot_w_legend(pd_df=pd.DataFrame(),
                      title='',
                      colors=None,
                      colors_in_df=True,
                      ax=None):
    
#     colors = pd_df.iloc[:,1] if colors is None else colors
    txts = pd_df.index
    if colors is None and colors_in_df is False:
#         if colors_in_df is True:
        patches = pd_df.iloc[:,0].plot(kind='pie',autopct='%.2f',labels=['' for _ in pd_df.iloc[:,0]], fontsize=0, ax=ax, figsize=[15,15], startangle=45)
    if colors_in_df is True:
        colors = pd_df.iloc[:,1]
        patches = pd_df.iloc[:,0].plot(kind='pie',autopct='%.2f',labels=['' for _ in pd_df.iloc[:,0]], fontsize=0, ax=ax, figsize=[15,15], colors=colors, startangle=45)
    plt.title(title)
    plt.ylabel('');
    labels = ['{1:0.2f} % - {0}'.format(i,j) for i,j in zip(txts, pd_df.iloc[:,0].divide(pd_df.iloc[:,0].sum())*100)]
    plt.legend(labels=labels, loc='best');

# Uso de series en API Series de Tiempo

In [9]:
## Datos analytics

In [10]:
# Cargo csv's
# df_analytics = read_files_to_df('/home/melik/sdt-analytics-download/')
df_analytics = read_files_to_df(analytics_tools.DIR_DATA_GR)

/Users/abenassi/anaconda/envs/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [11]:
# Filtro analytics por valores de interés
uri_georef = df_analytics.uri.str.contains('/georef/api/')
status_ok = df_analytics.status_code == 200
status_nan = df_analytics.status_code.isnull()

# df_analytics = df_analytics[(uri_serie) & (status_ok | status_nan)]
df_analytics = df_analytics[uri_georef & ~status_nan & status_ok]

In [12]:
# Filtro analytics. Saco ip's de test, saco períodos sin datos

# ips asociados al pico del 2018-12-21
test_ips = ['190.16.55.43','190.210.119.109', '190.246.123.181']
# ips asociados al pico del 2018-02-15
test_ips.extend(['195.162.12.14','190.18.52.25'])

exclude_test_ips = ~df_analytics.ip_address.isin(test_ips)
df_analytics = df_analytics[exclude_test_ips]

In [13]:
# Cambio el tipo de dato de la columna 'timestamp' a datetime
df_analytics['start_time_utc'] = pd.to_datetime(df_analytics['start_time'])

df_analytics['start_time'] = pd.DatetimeIndex(df_analytics['start_time']).tz_convert(tz='America/Argentina/Buenos_Aires')
df_analytics["date"] = pd.DatetimeIndex(df_analytics.start_time).normalize()
df_analytics["hour"] = pd.DatetimeIndex(df_analytics.start_time).hour
df_analytics["month"] = pd.DatetimeIndex(df_analytics.start_time).month
df_analytics["year"] = pd.DatetimeIndex(df_analytics.start_time).year
df_analytics['weekday'] = pd.DatetimeIndex(df_analytics.start_time).weekday
df_analytics['weekofyear'] = pd.DatetimeIndex(df_analytics.start_time).weekofyear
df_analytics['weekdate'] = df_analytics["date"].apply(get_week)
df_analytics['weekdate_short'] = df_analytics.weekdate.dt.strftime("%Y-%m-%d")
df_analytics['date_short'] = pd.DatetimeIndex(df_analytics.date).strftime('%Y-%m')

In [14]:
# agrego clasificación de qs's
qs_contains_name = df_analytics.querystring.str.contains('nombre=').astype(bool)
qs_contains_dir = df_analytics.querystring.str.contains('direccion=').astype(bool)
uri_is_ubicacion = df_analytics.uri.str.contains('ubicacion').astype(bool)

query_is_norm = (qs_contains_name & (~qs_contains_dir) & (~uri_is_ubicacion))
query_is_norm_dir = (qs_contains_dir & (~qs_contains_name) & (~uri_is_ubicacion))
query_is_ref = ~(query_is_norm | query_is_norm_dir | uri_is_ubicacion)

df_analytics.loc[uri_is_ubicacion, 'query_tipo'] = 'enriquecimiento'
df_analytics.loc[query_is_norm, 'query_tipo'] = 'normalizacion_nombres'
df_analytics.loc[query_is_norm_dir, 'query_tipo'] = 'normalizacion_direcciones'
df_analytics.loc[query_is_ref, 'query_tipo'] = 'referenciación'



In [15]:
# separo columna endpoint
df_analytics['endpoint'] = df_analytics.uri.apply(lambda x: str(x).strip('/').split('/')[-1].split('.')[0])

# reemplazo los methods que no son GET o POST por OTROS
# other_mr = ~df_analytics.request_method.isin(['GET','POST','HEAD'])
# df_analytics.loc[other_mr,'request_method'] = 'OTROS'


In [16]:
# df_analytics = df_analytics.merge(pd.get_dummies(df_analytics.query_tipo), right_index=True, left_index=True)

df_evolucion_by_tipo = df_analytics[['id','date_short','query_tipo']].groupby(['date_short','query_tipo']).count()

In [17]:
df_ips_by_tipo = df_analytics.groupby(['date_short','query_tipo'])['ip_address'].nunique().to_frame()

In [18]:
df_hits = df_analytics.groupby(['request_method','endpoint','query_tipo'])['id'].count()

df_hits = df_hits.to_frame().reset_index().rename({'id':'hits'}, axis=1)


In [19]:
# df_osde_by_month = df_log[df_log.http_referer.str.contains('osde')].time_local.apply(lambda x: str(x)[:7]).value_counts().to_frame().sort_index()

In [20]:
df_analytics[~df_analytics.request_method.isin(['GET','POST'])].request_method.fillna('nan').value_counts()

nan     296810
HEAD    45414 
Name: request_method, dtype: int64

In [21]:
df_analytics[~df_analytics.request_method.isin(['GET','POST','HEAD'])].date_short.value_counts().to_frame().sort_index()

,date_short
2018-07,2852
2018-08,41958
2018-09,108909
2018-10,125990
2018-11,17101
